In [18]:
pip install pandas fuzzywuzzy


In [28]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import pandas as pd

# Load data from CSV files
base_names_df = pd.read_csv('/content/sample_data/Book 3(Sheet1).csv')
variations_df = pd.read_csv('/content/sample_data/Book4(Sheet1).csv')



In [30]:
# Convert columns to string and handle NaN values
base_names_df['Base_Name'] = base_names_df['Base_Name'].astype(str)
variations_df['Variation'] = variations_df['Variation'].astype(str)


In [31]:
# Normalize function
def normalize(name):
    return ''.join(e for e in name.lower() if e.isalnum())

# Normalize base names and variations
base_names_df['Normalized_Name'] = base_names_df['Base_Name'].apply(normalize)
variations_df['Normalized_Variation'] = variations_df['Variation'].apply(normalize)

# Create a dictionary of base names
base_names_dict = base_names_df.set_index('Normalized_Name')['Base_Name'].to_dict()

# Function to find matches
def find_matches(variation, base_names_dict):
    normalized_variation = normalize(variation)
    matches = []
    for base_name in base_names_dict.values():
        score = fuzz.partial_ratio(normalized_variation, normalize(base_name))
        if score > 80:  # Adjust the threshold as needed
            matches.append((base_name, score))
    return matches

In [32]:

# Find matches for all variations
variations_df['Matches'] = variations_df['Variation'].apply(lambda x: find_matches(x, base_names_dict))

# Expand matches into a DataFrame for better readability
matches_expanded = variations_df.explode('Matches')

# Convert matches to separate columns
matches_expanded[['Matched_Base_Name', 'Score']] = pd.DataFrame(matches_expanded['Matches'].tolist(), index=matches_expanded.index)

# Drop the intermediate 'Matches' column
matches_expanded = matches_expanded.drop(columns=['Matches'])

# Print results
print(matches_expanded)

# Optionally save results to a CSV file
matches_expanded.to_csv('matched_names.csv', index=False)

        Variation Matches_With_Base_Name  Unnamed: 2  Unnamed: 3  \
0    Thomas  King            Thomas King         NaN         NaN   
1      ThomasKing            Thomas King         NaN         NaN   
2    Maria Garcia           Maria Garcia         NaN         NaN   
3       MaryLewis             Mary Lewis         NaN         NaN   
4        Nancy W.           Nancy Wright         NaN         NaN   
..            ...                    ...         ...         ...   
97       David M.         David Martinez         NaN         NaN   
98    Paul Allen.             Paul Allen         NaN         NaN   
99    Paul  Allen             Paul Allen         NaN         NaN   
100           nan                    NaN         NaN         NaN   
100           nan                    NaN         NaN         NaN   

    Normalized_Variation Matched_Base_Name  Score  
0             thomasking       Thomas King  100.0  
1             thomasking       Thomas King  100.0  
2            mariagarcia   